In [ ]:
!pip install -e /PlanRGCN/load_balance/

In [11]:
import pandas as pd
import numpy as np
import os
import importlib
import load_balance.workload as workload
import load_balance.query as query
import load_balance.arrival_time as at
importlib.reload(at)
importlib.reload(workload)
importlib.reload(query)
Query = query.Query
ArrivalRateDecider = at.ArrivalRateDecider
Workload = workload.Workload

In [12]:
df = pd.read_csv('/data/DBpedia2016_0_1_10_path_aug/train_sampled.tsv', sep='\t')

In [13]:
df.head(5)

,id,queryString,query_string_0,latency_0,resultset_0,query_string_1,latency_1,resultset_1,query_string_2,latency_2,...,Alternative,ComplexPath,MoreThanOnePredicate,queryID,Queries with 1 TP,Queries with 2 TP,Queries with more TP,S-P Concrete,P-O Concrete,S-O Concrete
0,http://lsq.aksw.org/lsqQuery-pMdmnYbKrDKIfHyUb...,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,0.017741,NaN,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,0.006007,NaN,PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-...,0.019850,...,0,0,0,http://lsq.aksw.org/lsqQuery-pMdmnYbKrDKIfHyUb...,1,0,0,0,0,0
1,http://lsq.aksw.org/lsqQuery-OxoJr-_LrTMGtMU90...,PREFIX xsd: <http://www.w3.org/2001/XMLSchema#...,PREFIX xsd: <http://www.w3.org/2001/XMLSchema#...,0.143848,NaN,PREFIX xsd: <http://www.w3.org/2001/XMLSchema#...,0.038889,NaN,PREFIX xsd: <http://www.w3.org/2001/XMLSchema#...,0.111012,...,0,0,0,http://lsq.aksw.org/lsqQuery-OxoJr-_LrTMGtMU90...,0,0,1,1,0,0
2,http://lsq.aksw.org/lsqQuery-eWXAkkV0HEhVJH54_...,PREFIX dby: <http://dbpedia.org/class/yago/> P...,PREFIX dby: <http://dbpedia.org/class/yago/> P...,0.020718,NaN,PREFIX dby: <http://dbpedia.org/class/yago/> P...,0.003342,NaN,PREFIX dby: <http://dbpedia.org/class/yago/> P...,0.002865,...,0,0,0,http://lsq.aksw.org/lsqQuery-eWXAkkV0HEhVJH54_...,0,0,1,0,1,0
3,http://lsq.aksw.org/lsqQuery-o1HoGA-SkHdgyIJa_...,SELECT DISTINCT ?property FROM <http://www.w3....,SELECT DISTINCT ?property FROM <http://www.w3....,0.011756,NaN,SELECT DISTINCT ?property FROM <http://www.w3....,0.002562,NaN,SELECT DISTINCT ?property FROM <http://www.w3....,0.025801,...,0,0,0,http://lsq.aksw.org/lsqQuery-o1HoGA-SkHdgyIJa_...,1,0,0,0,0,0
4,http://lsq.aksw.org/lsqQuery-A713d3YtNCeXejPPO...,PREFIX dbpo: <http://dbpedia.org/ontology/> PR...,PREFIX dbpo: <http://dbpedia.org/ontology/> PR...,0.007046,NaN,PREFIX dbpo: <http://dbpedia.org/ontology/> PR...,0.005886,NaN,PREFIX dbpo: <http://dbpedia.org/ontology/> PR...,0.003202,...,0,0,0,http://lsq.aksw.org/lsqQuery-A713d3YtNCeXejPPO...,0,0,1,0,0,0


df[df['id']=='http://lsq.aksw.org/lsqQuery-b_Tjy3b2j03Ab56KAZ-Hw-tFyANBndGDqUnHc0p50WM']

In [4]:
df['mean_latency'].quantile(q=0.25)

0.037649461068212944

In [5]:
w = Workload()
w.load_queries('/data/DBpedia2016_0_1_10_path_aug/train_sampled.tsv')
w.set_time_cls("/data/DBpedia2016_0_1_10_path_aug/planrgcn_binner/train_pred.csv")
a = ArrivalRateDecider()
w.set_arrival_times(a.assign_arrival_rate(w, mu=40))

loaded 16206


In [6]:
print(w[0])
w.shuffle_queries()
print(w[0])

(<load_balance.query.Query object at 0x7f6d2b13cd90>, 0.011731702249421478)
(<load_balance.query.Query object at 0x7f6d29df03a0>, 0.011731702249421478)


In [ ]:
print(w[:10])

In [8]:

from load_balance.timer import Timer

from load_balance.worker import Worker
        
from load_balance.dat_struct import Queue

from load_balance.balancer import Balancer

            
b = Balancer(w)     
b.run()      

In [ ]:
print(b.slow_queue.worker[0].past_queries[0])

In [10]:
def get_all_queries(b):
    queries = []
    for w in b.slow_queue.worker:
        queries.extend(w.past_queries)
    for w in b.med_queue.worker:
        queries.extend(w.past_queries)
    for w in b.fast_queue.worker:
        queries.extend(w.past_queries)
    return queries
def workload_latency(qs):
    total_latency = 0
    for q in qs:
        total_latency += (q.finish_time- q.arrivaltime)
    return total_latency
qs = get_all_queries(b)
workload_latency(qs)/len(qs)

202.51363121171752

In [ ]:
len(qs)

In [ ]:
q = qs[0]
print(q.arrivaltime, q.finish_time, q.execution_time, q.starttime)
